# Comparison Notebook

In [1]:
import torch
import time
import os
import requests
import tempfile
import numpy as np
import onnxruntime as ort
from transformers import AutoImageProcessor, SuperPointForKeypointDetection
from models.patchmatch_triplet_descriptor import PatchMatchTripletNetwork, PatchMatchEncoder
from PIL import Image

/home/rehan/anaconda3/envs/patchmatch++/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
2025-07-15 21:31:44.608322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752597105.103010  214210 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752597105.249455  214210 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752597106.816587  214210 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752597106.816616  214

In [3]:
# --- Load a single image from URL ---
def load_image(url):
    return Image.open(requests.get(url, stream=True).raw).convert("RGB")

image = load_image("http://images.cocodataset.org/val2017/000000039769.jpg")
image_resolution = (image.height, image.width)

# --- Load processor and model ---
processor = AutoImageProcessor.from_pretrained("magic-leap-community/superpoint")
model = SuperPointForKeypointDetection.from_pretrained("magic-leap-community/superpoint")
model.eval()

# --- Move to CPU ---
device = torch.device("cpu")
model.to(device)

# --- Measure model size (MB) ---
def get_model_size(model):
    with tempfile.NamedTemporaryFile(delete=False) as tmp:
        torch.save(model.state_dict(), tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024 * 1024)
    os.remove(tmp.name)
    return round(size_mb, 2)

# --- Count total number of parameters ---
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

model_size_mb = get_model_size(model)
param_count = count_parameters(model)

# --- Preprocess single image ---
inputs = processor([image], return_tensors="pt").to(device)

# --- Inference timing ---
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_time = time.time()

inference_time_ms = (end_time - start_time) * 1000

# --- Post-process output ---
image_size = [image_resolution]
output = processor.post_process_keypoint_detection(outputs, image_size)[0]
num_descriptors = output["descriptors"].shape[0]

# --- Print Results ---
print("\n✅ SuperPoint Evaluation on Single Image")
print(f"🖼️  Image Resolution        : {image_resolution[0]} x {image_resolution[1]}")
print(f"📦 Model Size               : {model_size_mb} MB")
print(f"🔢 Model Parameters         : {param_count:,}")
print(f"⏱️  Inference Time (CPU)     : {inference_time_ms:.2f} ms")
print(f"#️⃣  Descriptors Extracted   : {num_descriptors}")


✅ SuperPoint Evaluation on Single Image
🖼️  Image Resolution        : 480 x 640
📦 Model Size               : 4.97 MB
🔢 Model Parameters         : 1,300,865
⏱️  Inference Time (CPU)     : 547.33 ms
#️⃣  Descriptors Extracted   : 556


In [30]:
# --- Configuration ---
MODEL_PATH = "../model_outputs/patchmatch_model.onnx"
PATCH_SHAPE = (40, 40)
NUM_PATCHES = 550
EMBED_DIM = 128

# --- Generate dummy input ---
dummy_patches = np.random.rand(NUM_PATCHES, 1, *PATCH_SHAPE).astype(np.float32)

# --- Load ONNX session (CPU inference) ---
session = ort.InferenceSession(MODEL_PATH, providers=["CPUExecutionProvider"])
input_name = session.get_inputs()[0].name

# --- Inference timing ---
start_time = time.time()
output = session.run(None, {input_name: dummy_patches})
end_time = time.time()

# --- Output shape ---
output_tensor = output[0]
embedding_shape = output_tensor.shape

# --- Model size ---
model_size_mb = os.path.getsize(MODEL_PATH) / (1024 * 1024)

# --- Time in ms ---
inference_time_ms = (end_time - start_time) * 1000

# --- Print results ---
print("\n✅ PatchMatch ONNX Benchmark (CPU)")
print(f"📦 ONNX Model Size         : {model_size_mb:.2f} MB")
print(f"📤 Input Patches           : {NUM_PATCHES} x {PATCH_SHAPE[0]} x {PATCH_SHAPE[1]}")
print(f"📥 Output Embedding Shape  : {embedding_shape}")
print(f"⏱️  Inference Time (CPU)    : {inference_time_ms:.2f} ms")


✅ PatchMatch ONNX Benchmark (CPU)
📦 ONNX Model Size         : 0.38 MB
📤 Input Patches           : 550 x 40 x 40
📥 Output Embedding Shape  : (550, 128)
⏱️  Inference Time (CPU)    : 1120.85 ms
